In [1]:
from Linguo_tools import *
import nltk
from tqdm import tqdm
import math
import random

exp = Experimenter()
labeled_gram,labeled_ws = exp.load_corpora_from_file("Data/hundo_gram","Data/hundo_trigramWS")
datasplit = int(math.floor(0.8*len(labeled_gram)))
training_gram = [(sent,label) for sent,label in labeled_gram[:datasplit]]
testing_gram = [(sent,label) for sent,label in labeled_gram[datasplit:]]

training_ws = [(sent,label) for sent,label in labeled_ws[:datasplit]]
testing_ws = [(sent,label) for sent,label in labeled_ws[datasplit:]]


In [2]:
trainingData = training_gram + training_ws
random.shuffle(trainingData)
model = exp.train_model(trainingData)

  0%|          | 0/149394 [00:00<?, ?it/s]

Epoch 1


  0%|          | 4/149394 [00:00<1:15:04, 33.17it/s]

	 Epoch0:38390.39437574822
Epoch 2


  0%|          | 0/149394 [00:00<?, ?it/s]

	 Epoch1:20575.83715442849
Epoch 3


100%|██████████| 149394/149394 [1:06:17<00:00, 37.56it/s]

	 Epoch2:17242.71104878809


In [5]:
test_data = testing_gram + testing_ws
random.shuffle(test_data)
results = exp.test_model(test_data, model)

In [7]:
def test_model_collect_errors(exp, test_data, model):
        correct = 0.0
        tp = 0.0
        tn = 0.0
        fp = 0.0
        fn = 0.0
        false_negatives = []
        false_positives = []
        print("Begining test")
        for testcase in tqdm(test_data):
            target = testcase[1]
            prepared_inputs = exp.prepare_input(testcase[0])
            prediction_vec = model(prepared_inputs).view(2)
            if prediction_vec.data[0] > prediction_vec.data[1]:
                prediction = 0
            else:
                prediction = 1
            if prediction == testcase[1]:
                correct += 1
                if target == 1:
                    tp += 1
                else:
                    tn += 1
            else:
                if target == 1:
                    fn += 1
                    false_negatives.append(" ".join(testcase[0]))
                else:
                    fp += 1
                    false_positives.append(" ".join(testcase[0]))

        # Compile results
        accuracy = correct/len(test_data)
        precision = tp / (tp+fp)
        recall = tp / (tp+fn)
        fmeasure = 2*tp / (2*tp+fp+fn)
        results = {"accuracy": accuracy,
                   "precision": precision,
                   "recall": recall,
                   "fmeasure": fmeasure,
                   "tp": tp,
                   "tn": tn,
                   "fp": fp,
                   "fn": fn}
        return results, false_negatives,false_positives

In [9]:
results, false_neg, false_pos = test_model_collect_errors(exp,test_data, model)

  0%|          | 12/37349 [00:00<05:21, 116.02it/s]

Begining test


100%|██████████| 37349/37349 [04:04<00:00, 152.99it/s]


In [10]:
false_neg

['mis problemas comenzaron temprano esta mañana cuando se canceló mi vuelo de belfast a londres .',
 'aprovecho esta oportunidad para decir que apruebo el programa en el contexto del plan estratégico quinquenal .',
 'proyectos como el del túnel de base del brenner solamente pueden ser competitivos si el transporte de mercancías se puede ofrecer a un precio capaz de competir .',
 'apoyamos la petición relativa a la simplificación de los procedimientos de gestión de meda y el fortalecimiento de su enfoque integrado y estratégico .',
 'abre las puertas a un mercado mundial , así como a información prácticamente ilimitada a todos aquellos que tengan acceso a una pantalla de ordenador .',
 'casi la mitad de los 127 millones de habitantes que componen su población vive por debajo del umbral de la pobreza .',
 'algo tiene que cambiar .',
 'no .',
 'lamentamos que inicialmente hubiese cierta resistencia por parte del consejo pero nos complace mucho que , como resultado de la efectiva intervenc

In [11]:
false_pos

['repito a su ausencia aquí es pedir que hagamos en la cuenta bancaria sin la autorización , para exponer un marco .',
 'cualquier solicitante de subvenciones a través de esta catástrofe natural como el descrito demuestran su voluntad constructiva , equilibrada y muy deficiente en un pilar y estoy bastante sorprendido ante las opiniones políticas y directrices concretas proporcionando .',
 'va dirigido al #unk coronel blanco , donde &quot; sin la necesaria subsidiariedad y sigo convencido de que éstos supuestamente envían a los vehículos .',
 'tengo aquí solicitudes de pago de compensaciones internas y de telecomunicaciones en la respuesta sea &apos; bueno , esa imagen de marca , esa especie de derecho constituye la esencia fundamental de respetar estos valores fundamentales están reconocidos en los actuales .',
 'balanza comercial , mercantil , y tenemos que ajustarnos a las ponentes por un propietario francés de nuestros prójimos en refugiados en nuestro triángulo institucional .',
 